# Liner models

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ChemAI-Lab/Math4Chem/blob/main/website/Lecture_Notes/Notes/Coding/nonlinear_equations.ipynb)


**References:**
1. **Chapters 1 and 3**: Pattern Recognition and Machine Learning, C. M. Bishop.

## Linear Models
The simplest models for regression, are the ones that involved a linear combination of input variables,
$$
f(\mathbf{x},\mathbf{w}) = \underbrace{\mathbf{w}^\top \mathbf{x}}_{\text{dot product}}= \sum_{i}^d w_i \, x_i,
$$
where $\mathbf{w}$ are the linear weights, also known as **parameters**, and $\mathbf{x}$ is the input. $d$ is known as the number of features that represent $\mathbf{x}$. <br>

The family of "linear models" is not unique, it can be extended to **fixed** non-linear functions, 
$$
f(\mathbf{\phi}(\mathbf{x}),\mathbf{w}) = \mathbf{w}^\top \mathbf{\phi}(\mathbf{x})= \sum_{i}^d w_i \, \phi_i(\mathbf{x}),
$$
where $\phi_i(\mathbf{x})$ is a "basis function".



In [ ]:
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as f_mse

import matplotlib.pyplot as plt

In [ ]:
# generate random data over f(x) = sin(x) + x - 1
def get_data(N, bool_biased=True):
    # This creates an array x of N linearly spaced values between -1 and 1.
    # bool_biased: if True, adds a bias term (column of ones) to the input data X

    x = np.linspace(-1., 1., N) + np.random.uniform(low=-.1, high=.1, size=N)
    y = 1.2*np.sin(2*x) + x - 1.
    # Adds random noise to each y value.
    y = y + np.random.uniform(low=-.35, high=.35, size=x.shape)
    if bool_biased:
        X = np.column_stack((np.ones_like(x), x))
    else:
        X = x[:, None]
    return X, y

In [ ]:
X, y = get_data(15, bool_biased=True)

plt.plot(X[:, 1], y, 'o', label='Data')
plt.xlabel('x', fontsize=16)
plt.ylabel('f(x)', fontsize=16)
plt.legend()

## Least Square Problem
**Mean Square Error**
$$
{\cal L}(\mathbf{w}) = \frac{1}{2n}\sum_{i}^{n} (\hat{y}_i \mathbf{w}^\top\phi(\mathbf{x}_i)) = \frac{1}{2n} \left (\mathbf{y} - \Phi(\mathbf{X})\mathbf{w} \right)^\top \left (\mathbf{y} - \Phi(\mathbf{X})\mathbf{w} \right)
$$

**Gradient of a function equal to zero**  
$$
    \nabla {\cal L}(\mathbf{w}) \Big\rvert_{\mathbf{w}^{*}} = \frac{1}{2n} \nabla_{\mathbf{w}} \left [ \left (\mathbf{y} - \Phi(\mathbf{X})\mathbf{w} \right)^\top \left (\mathbf{y} - \Phi(\mathbf{X})\mathbf{w} \right) \right ]= 0
$$

To solve for $\mathbf{w}^*$, let's expand $ \left (\mathbf{y} - \Phi(\mathbf{X})\mathbf{w} \right)^\top \left (\mathbf{y} - \Phi(\mathbf{X})\mathbf{w} \right)$,

$$
    \left (\mathbf{y} - \Phi(\mathbf{X})\mathbf{w} \right)^\top \left (\mathbf{y} - \Phi(\mathbf{X})\mathbf{w} \right) = \mathbf{y}^\top \mathbf{y}  - \mathbf{y}^\top \Phi(\mathbf{X})\mathbf{w} -  \mathbf{w}^\top\Phi(\mathbf{X})^\top\mathbf{y} +   \mathbf{w}^\top\Phi(\mathbf{X})^\top \Phi(\mathbf{X})\mathbf{w}
$$
$$
    \nabla_{\mathbf{w}} {\cal L}(\mathbf{w}) = \frac{1}{2n}\left(  -2 \Phi(\mathbf{X})^\top\mathbf{y} + 2\Phi(\mathbf{X})^\top\Phi(\mathbf{X})\mathbf{w} \right) = 0
$$

**Optimal parameters**

$$
 \mathbf{w}^* = \left ( \Phi(\mathbf{X})^\top \Phi(\mathbf{X}) \right ) ^{-1} \Phi(\mathbf{X})^\top \mathbf{y}
$$

$\,$<br>
**Notes from CHEM-3PC3**: Least Square:  [![Download PDF](https://img.shields.io/badge/Download_PDF-Click_Here-blue.svg)](https://github.com/ChemAI-Lab/Math4Chem/raw/main/website/Lecture_Notes/Notes/Linear_Regression.pdf)  <br>
 
**Extra:**
1. Homework, Proof the above equations.
2. [Equations from Sections 2.4.1 and 2.4.2](https://www2.imm.dtu.dk/pubdb/edoc/imm3274.pdf)

In [ ]:
# trining a linear model
def linear_model_solver(X, y):
    Xt = X.T  # transpose of X
    A = Xt @ X  # A = X^T X
    z = Xt @ y  # z = X^T y
    A_inv = np.linalg.inv(A)  # inverse of A
    w_opt = A_inv @ z  # w = A^-1 z
    return w_opt  # optimal parameters

In [ ]:
# find the optimal parameters
w_opt = linear_model_solver(X, y)
print("Optimal parameters")
for i,wi in enumerate(w_opt):
    print(f"w_{i} = {wi:.3f}, ", end='')
print( )

y_data_pred = X@w_opt
r2_value = r2_score(y,y_data_pred)
print(f'R-squared score: {r2_value:.3f}')

In [ ]:
# plot our model
x_grid = np.linspace(-1.1, 1.1, 100)
X_grid = np.column_stack((np.ones_like(x_grid), x_grid))  # add bias term
print("Grid with bias: ", X_grid.shape)
y_pred = X_grid@w_opt

# plot the data
fig, ax = plt.subplots(figsize=(11, 8))
ax.text(0.05, 0.8, r'$R^2 = $' +
        f'{r2_value:0.3f}', transform=ax.transAxes, fontsize=20)
ax.scatter(X[:, 1], y, label='data', s=75)
ax.plot(X_grid[:, 1], y_pred, c='k', label='Linear model', markersize=5)
ax.set_xlabel('x', fontsize=18)
ax.set_ylabel('f(x)', fontsize=18)
plt.legend()

## Beyond Linear Models
Instead of considering for each data point, $\mathbf{x}^\top = [1, x]$, we can assume any other representation using a "basis set" $\phi$, for example, a polynomial one, 
This is simply a new representation of $x$
$$
\phi(\mathbf{x})^\top = [1, x, x^2, x^3, \cdots,x^p].
$$

Instead of coding how to create the polynomial representation, we will use Sklearn. <br>
Read the documentation: [`PolynomialFeatures`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html)

In [ ]:
def polynomial_features(X, p):
    # transform the input data to polynomial features up to degree p
    poly = PolynomialFeatures(p)
    Phi = poly.fit_transform(X)
    return Phi

### Extra: Gaussian (RBF) Feature Expansion
We can also expand features using a Gaussian (radial basis) function.


In [ ]:
from sklearn.kernel_approximation import RBFSampler
from scipy.spatial.distance import cdist

def gaussian_features(X, n_gaussians, length_scale=0.2):
    # transform the input data to Gaussian basis functions
    # X: input data (N x D)
    # n_gaussians: number of Gaussian basis functions
    # length_scale: length scale of the Gaussians

    # Use the raw x values (without the bias column)
    x_min, x_max = np.min(X[:, 1]), np.max(X[:, 1])
    x_grid_features = np.linspace(x_min, x_max, n_gaussians)[:, None]

    dists = cdist(X[:, 1:] / length_scale, x_grid_features /
                  length_scale, metric="sqeuclidean")
    Phi = np.exp(-0.5 * dists)
    return Phi

In [ ]:
# transform the training data to polynomial features
p = 3  # degree

X = X[:, -1:]  # select the last column only (without bias term)

X_phi = polynomial_features(X, p)
print(X.shape, X_phi.shape)
w_phi_opt = linear_model_solver(X_phi, y)
print("Optimal parameters")
for i, wi in enumerate(w_phi_opt):
    print(f"w_{i} = {wi:.3f}, ", end='')
print()

y_data_phi_pred = X_phi@w_phi_opt
r2_score_poly = r2_score(y, y_data_phi_pred)
print(f'R2 polynomial model: {r2_score_poly:.3f}')

X_grid = np.linspace(-1.25, 1.25, 100)[:, None]
X_grid_phi = polynomial_features(X_grid, p)
y_pred_phi = X_grid_phi@w_phi_opt

# plot the data and the polynomial model
fig, ax = plt.subplots(figsize=(11, 8))
ax.scatter(X[:, 0], y, label='data', s=75)
ax.plot(X_grid[:, 0], y_pred, c='green', ls='--',
        label='linear model ' + r'$R^2 = $' +
        f'{r2_value:0.3f}', markersize=5)
ax.plot(X_grid[:, 0], y_pred_phi, c='k',
        label=f'polynomial model (p={p}), R2 = {r2_score_poly:.3f}', markersize=5)
ax.set_xlabel('x', fontsize=18)
ax.set_ylabel('f(x)', fontsize=18)
plt.legend(fontsize=14)

In [ ]:
# R2 as a function of the polynomial degree

fig, ax = plt.subplots(figsize=(11, 8))
ax.scatter(X[:, 0], y, label='data', s=105,zorder=4)
    
for p in range(1,11,2):# loop over polynomial degrees p = [2, 4, .., 10]
    X_phi = polynomial_features(X, p)
    w_phi_opt = linear_model_solver(X_phi, y) # we can reuse our function
    # print(f'Optimal parameters\n{w_phi_opt}')

    y_data_phi_pred = X_phi@w_phi_opt
    r2_score_poly = r2_score(y, y_data_phi_pred)
    print(f'p: {p}, R2= {r2_score_poly:.4f}')

    X_grid_phi = polynomial_features(X_grid, p)
    y_pred_phi = X_grid_phi@w_phi_opt
# plot the data and the polynomial model

    ax.plot(X_grid[:, 0], y_pred_phi,
            label=f'p={p} ' +  r' $R^2 = $' +
            f'{r2_score_poly:0.4f}',  markersize=5, alpha=0.95*p/15)
ax.set_xlabel('x', fontsize=18)
ax.set_ylabel('f(x)', fontsize=18)
ax.set_ylim(-5,3)
plt.legend(fontsize=10,loc=0)

**How can we select the best model?**

In [ ]:
X, y = get_data(25, bool_biased=True)
X_tr,X_val, y_tr,y_val = train_test_split(X,y,test_size=0.2, random_state=42)
print("Training data:", X_tr.shape, y_tr.shape)
print("Validation data:", X_val.shape, y_val.shape)

In [ ]:
for p in range(1,11,2):
    # transform the training data to polynomial features
    X_tr_phi = polynomial_features(X_tr, p)
    w_phi_opt = linear_model_solver(X_tr_phi, y_tr)

    # evaluate on training data
    y_tr_phi_pred = X_tr_phi@w_phi_opt
    r2_score_tr = r2_score(y_tr, y_tr_phi_pred)
    mse_tr = f_mse(y_tr, y_tr_phi_pred)
    rmse_tr = np.sqrt(mse_tr)

    # transform the validation data to polynomial features
    X_val_phi = polynomial_features(X_val, p)
    y_val_phi_pred = X_val_phi@w_phi_opt
    r2_score_val = r2_score(y_val, y_val_phi_pred)
    mse_val = f_mse(y_val, y_val_phi_pred)
    rmse_val = np.sqrt(mse_val)

    print(f'p: {p}, R2 train= {r2_score_tr:.4f}, R2 val= {r2_score_val:.4f}', end=', ')
    print(f'         RMSE train= {rmse_tr:.4f}, RMSE val= {rmse_val:.4f}')

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ChemAI-Lab/Math4Chem/blob/main/website/Lecture_Notes/Notes/Coding/nonlinear_equations.ipynb)

Topics
1. Linear regression and least square problem
   1. Overfitting with Polynomials (1D problem for example)
2. Ridge regression to avoid Over-fitting 